In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.options.display.max_columns = 500

### Загрузим датасет с машинами. Цель - верно восстанавливать для каждой из них цену продажи!

In [2]:
data = pd.read_csv('autos.csv')

data.head()

,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner
0,Maruti 800 AC,2007,60000,70000,Petrol,Individual,Manual,First Owner
1,Maruti Wagon R LXI Minor,2007,135000,50000,Petrol,Individual,Manual,First Owner
2,Hyundai Verna 1.6 SX,2012,600000,100000,Diesel,Individual,Manual,First Owner
3,Datsun RediGO T Option,2017,250000,46000,Petrol,Individual,Manual,First Owner
4,Honda Amaze VX i-DTEC,2014,450000,141000,Diesel,Individual,Manual,Second Owner


In [3]:
### Колонка с тергетом - "selling price"

X = data.drop("selling_price", axis=1)
y = data["selling_price"]

### Будем замерять MSLE!
### Поэтому прологарифмируем таргет
### А после оптимизируем MSE

y = y.apply(np.log1p)

In [4]:
### Разделим выборку на трейн и тест!

from sklearn.model_selection import train_test_split 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [5]:
data.isna().sum()

name             0
year             0
selling_price    0
km_driven        0
fuel             0
seller_type      0
transmission     0
owner            0
dtype: int64

In [6]:
data.shape

(4340, 8)

__Задание__ 

Реализуйте свой MeanTargetEncoder с добавленем некоторого шума!

Однажды в лекционном материале, обсуждая счетчики, мы говорили с вами о том, что из-за них модели могут переобучаться. Один из способов бороться с этим - валидировать расчеты среднего таргета (стратегия отложенной выборки / расчеты на кросс-валидации). Но есть еще проще!

Можно просто к значению счетчика добавить случайный шум (зашумить данные)!

Напомним, что рассчитываться новые признаки должны по такой формуле:

$$
g_j = \frac{\sum_{i=1}^{l} [f_j(x) = f_j(x_i)]}{l} + C * \epsilon
$$


Пусть шум будет случайной величиной из нормального стандартного распределения, то есть $\epsilon \sim N(0, 1) $, а $ C = 0.006$.

Создавай свой класс-трансформер, наследуйтесь от классов `BaseEstimator, TransformerMixin` из `sklearn.base`. Трансформер не должен модифицировать передаваемую ему выборку inplace, а все необходимые статистики нужно считать только по обучающей выборке в методе `fit`. 
Ваш трансформер должен принимать при инициализации список из категориальных признаков и список из числовых признаков. 

Если для какого-то признака в тестовой выборке отсутствует значение, трансформер должен поставить там 0.

На выходе должен получиться датасет того же размера с измененными категориальными признаками

In [7]:
object_cols = ['name', 'year', 'fuel', 'seller_type', 'transmission', 'owner']
num_cols = ['km_driven']

X.head()

,name,year,km_driven,fuel,seller_type,transmission,owner
0,Maruti 800 AC,2007,70000,Petrol,Individual,Manual,First Owner
1,Maruti Wagon R LXI Minor,2007,50000,Petrol,Individual,Manual,First Owner
2,Hyundai Verna 1.6 SX,2012,100000,Diesel,Individual,Manual,First Owner
3,Datsun RediGO T Option,2017,46000,Petrol,Individual,Manual,First Owner
4,Honda Amaze VX i-DTEC,2014,141000,Diesel,Individual,Manual,Second Owner


In [8]:
data.nunique()

name             1491
year               27
selling_price     445
km_driven         770
fuel                5
seller_type         3
transmission        2
owner               5
dtype: int64

In [9]:
from sklearn.base import BaseEstimator, TransformerMixin

class MeanTargetEncoderNoise(BaseEstimator, TransformerMixin):
    
    def __init__(self, categorical, numeric):
        self.categorical = categorical
        self.numeric = numeric

    def fit(self, X, y):
        fit = X.copy()
        fit['target'] = y.copy()
        self.dict_of_mean = {col: fit.groupby(col)['target'].mean() for col in self.categorical}
        return self
        
    def transform(self, df):
        temp = df.copy()
        for col in self.dict_of_mean:
            temp[col] = temp[col].map(self.dict_of_mean[col])
            temp.loc[(temp[col].isna()), col] = 0
        return temp

In [10]:
### Проверка работы трансформера

np.random.seed(1)
transformer = MeanTargetEncoderNoise(categorical=object_cols, numeric=num_cols)

transformer.fit(X_train, y_train)

train = transformer.transform(X_train)
test = transformer.transform(X_test)

test

,name,year,km_driven,fuel,seller_type,transmission,owner
2761,13.667011,13.552865,15000,13.089123,12.618096,12.638189,12.972935
3210,13.040162,12.928875,70000,13.089123,12.618096,12.638189,12.460189
2606,13.122562,12.626962,90000,13.089123,12.618096,12.638189,12.972935
1030,12.384603,13.135543,41000,12.453348,13.147319,12.638189,12.972935
3942,13.378028,12.928875,71318,13.089123,13.147319,12.638189,12.972935
...,...,...,...,...,...,...,...
1206,12.146842,12.417709,80000,12.453348,12.618096,12.638189,12.460189
3167,12.611541,12.860755,90000,13.089123,12.618096,12.638189,12.460189
1297,0.000000,12.156812,60000,13.089123,12.618096,12.638189,12.121488
2188,13.446682,13.057348,66521,12.453348,13.640843,12.638189,12.972935


Обучите несколько деревьев, перебирая максимальную глубину алгоритма из списка `max_depth_list`, а остальные параметры оставьте дефолтными. Выведите лучшее значение гиперпараметра. Постройте график зависимости MSLE на тестовой выборке от значения гиперпараметра. Воспользуйтесь `Pipeline` без `GridSearch`. Проделайте то же самое с `min_samples_split`, `min_impurity_decrease`, `max_leaf_nodes`. (по 2б на каждый параметр)

In [11]:
train.isna().sum()

name            0
year            0
km_driven       0
fuel            0
seller_type     0
transmission    0
owner           0
dtype: int64

In [12]:
max_depth_ = [3, 5, 8, 12]
min_samples_split_ = [10, 50, 100, 500]
min_impurity_decrease_ = [0, 0.1, 0.15, 0.2]
max_leaf_nodes_ = [100, 200, 500]

In [13]:
from sklearn.metrics import mean_squared_error as mse
from sklearn.tree import DecisionTreeRegressor
from sklearn.pipeline import Pipeline

np.random.seed(1)

### Your code is here
def find_best_params_independent(model, hyperparameters, values):
    best_parameters = {}
    for i, hyperparameter in enumerate(hyperparameters):
        errors = []
        for value in values[i]:
            model.fit(train, y_train)
            error = np.mean((y_test - model.predict(test))**2)
            errors.append(error)
        best_parameters[hyperparameter] = values[i][errors.index(min(errors))]
    best = DecisionTreeRegressor(**best_parameters)
    best.fit(train, y_train)
    return best_parameters, f'with error = {np.mean((y_test - best.predict(test))**2)}'
    



Подберите лучшую комбинацию параметров, используя `GridSearchCV` и набор массивов значений параметров из предыдущего задания. Для лучшей комбинации посчитайте MSLE на тестовой выборке. Получились ли лучшие параметры такими же, как если бы вы подбирали их по-отдельности при остальных гиперпараметрах по умолчанию (предыдущее задание)? (2б)

In [14]:
forest = DecisionTreeRegressor()
hyperparams=['max_depth', 'min_samples_split', 'min_impurity_decrease', 'max_leaf_nodes']
values = [max_depth_, min_samples_split_, min_impurity_decrease_, max_leaf_nodes_]

In [15]:
find_best_params_independent(model=forest, hyperparameters=hyperparams, values=values)

({'max_depth': 8,
  'min_samples_split': 100,
  'min_impurity_decrease': 0,
  'max_leaf_nodes': 200},
 'with error = 0.9527565444305324')

In [16]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
param_grid = {
    "max_depth": [3, 5, 8, 12],
    "min_samples_split": [10, 50, 100, 500],
    "min_impurity_decrease": [0, 0.1, 0.15, 0.2],
    "max_leaf_nodes": [100, 200, 500]
}
np.random.seed(1)

def loss(estimator, X, y):
    return np.mean((y - estimator.predict(X))**2)
### Your code is here
forest = DecisionTreeRegressor()
gd = GridSearchCV(estimator=forest, param_grid=param_grid, scoring=loss)
gd.fit(train, y_train)




GridSearchCV(estimator=DecisionTreeRegressor(),
             param_grid={'max_depth': [3, 5, 8, 12],
                         'max_leaf_nodes': [100, 200, 500],
                         'min_impurity_decrease': [0, 0.1, 0.15, 0.2],
                         'min_samples_split': [10, 50, 100, 500]},
             scoring=<function loss at 0x13d7b25e0>)

In [17]:
np.mean((y_test - gd.predict(test))**2)

0.5204871413036717